In [1]:
import h5py
import numpy as np
import os,random
from keras.layers import Input,Reshape,ZeroPadding2D,Conv2D,Dropout,Flatten,Dense,Activation,MaxPooling2D,AlphaDropout,Conv1D,MaxPooling1D,BatchNormalization
from keras import layers,regularizers
import keras.models as Model
from keras.regularizers import *
from keras.optimizers import adam
import keras
import keras.backend.tensorflow_backend as tfback
import tensorflow as tf
import gc
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
from keras.callbacks import ReduceLROnPlateau
from sklearn.decomposition import PCA

Using TensorFlow backend.


In [5]:
#loading data
f = h5py.File("../modulation-prediction/data.hdf5", 'r')
X_train = np.array(f['train'])
# X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], X_train.shape[1])
# X_train = X_train.reshape(X_train.shape[0], 2*X_train.shape[1]) # PCA
print(X_train.shape)

X_test = np.array(f['test'])
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], -1)
# X_test = X_test.reshape(X_test.shape[0], 2*X_test.shape[1])
print(X_test.shape)

Y_train = pd.read_csv('../modulation-prediction/train_labels.csv').to_numpy()

(30000, 1024, 2)
(20000, 1024, 2)


In [ ]:
# PCA
# pca = PCA(n_components = 1024).fit(X_train.reshape(X_train.shape[0], 2*X_train.shape[1]))
# X_PCA = pca.tranform(X_train)
# X_PCA = X_PCA.reshape(X_PCA.shape[0], 2, X_PCA.shape[1]/2)

In [ ]:
class_index = {
 'FM': 0,
 'OQPSK':1,
 'BPSK':2,
 '8PSK':3,
 'AM-SSB-SC':4,
 '4ASK':5,
 '16PSK':6,
 'AM-DSB-SC':7, 
 'QPSK': 8, 
 'OOK':9
}

classnum = 10
def classToIndex(catg):
    return class_index[catg]

#direct mapping, not one-hot
def setToNum(dataset):
    output = np.zeros((dataset.shape[0], 1),dtype = int)
    for li, catg in enumerate(dataset[:,1]):
        output[li] = classToIndex(catg)
    return output         

In [ ]:
Y_number = setToNum(Y_train)
print(Y_number.dtype)

Y_train = keras.utils.to_categorical(Y_number, num_classes=classnum, dtype='int64')
print(Y_train.shape)

In [ ]:
"""Model"""
classes = [
 'FM',
 'OQPSK',
 'BPSK',
 '8PSK',
 'AM-SSB-SC',
 '4ASK',
 '16PSK',
 'AM-DSB-SC',
 'QPSK',
 'OOK']
data_format = 'channels_last'
initializer = 'glorot_normal'

def conv(Xm,Seq):
    # 3*3 Conv
    Xm = Conv1D(64, 3, padding='same', name=Seq+"_conv",activation = 'relu',kernel_initializer= initializer,data_format=data_format)(Xm)
#     Xm = BatchNormalization()(Xm)
    # ReLU
    Xm = Activation("relu")(Xm)
#     Xm = Dropout(0.1)(Xm)
    # MaxPooling
    Xm = MaxPooling1D(pool_size=2, strides=2, padding='valid',data_format=data_format)(Xm)
    return Xm


in_shp = (1024,2)#X_train.shape[1:]   # each sample is [1024,2]
#input layer
Xm_input = Input(in_shp)
Xm = Xm_input

#Conv Stack
Xm = conv(Xm,Seq = 'Conv1')   #shape:(64 * 512)
Xm = conv(Xm,Seq = 'Conv2')   #shape:(64 * 256)
Xm = conv(Xm,Seq = 'Conv3')   #shape:(64 * 128)
Xm = conv(Xm,Seq = 'Conv4')   #shape:(64 * 64)
Xm = Dropout(0.2)(Xm)
Xm = conv(Xm,Seq = 'Conv5')   #shape:(64 * 32)
Xm = Dropout(0.2)(Xm)
Xm = conv(Xm,Seq = 'Conv6')   #shape:(64 * 16)
Xm = Dropout(0.2)(Xm)
Xm = conv(Xm,Seq = 'Conv7')   #shape:(64 * 8)
Xm = Dropout(0.2)(Xm)
#Full Con 1
Xm = Flatten(data_format=data_format)(Xm)
Xm = Dense(128, activation='selu', kernel_regularizer=l2(0.0), kernel_initializer=initializer, name="dense1")(Xm)
Xm = AlphaDropout(0.3)(Xm)
#Full Con 2
Xm = Dense(128, activation='selu', kernel_regularizer=l2(0.0), kernel_initializer=initializer, name="dense2")(Xm)
Xm = AlphaDropout(0.25)(Xm)
#Full Con 3
Xm = Dense(len(classes), activity_regularizer=l2(0.000),activation = 'softmax',kernel_initializer=initializer, name="dense3")(Xm)
# Xm = AlphaDropout(0.2)(Xm)
#SoftMax
# Xm = Activation('softmax')(Xm)
#Create Model
model = Model.Model(inputs=Xm_input,outputs=Xm)
adam = keras.optimizers.Adam(lr=0.001)
# sgd = keras.optimizers.SGD(lr = 0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[keras.metrics.categorical_accuracy])
model.summary()

In [ ]:
# train
filepath = '../modulation-prediction/CNN_1d.h5'
# reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5,
#                               patience=5, min_lr=0)
hist = model.fit(X_train,
    Y_train,
    batch_size=32,
    epochs=1000,
    verbose=2,
    validation_split = 0.2,
    shuffle = True,
    callbacks = [
        keras.callbacks.ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=0, save_best_only=True, mode='auto'),
#         keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=10, verbose=0, mode='auto'),
#         reduce_lr
    ])

model.load_weights(filepath)

In [9]:
index_class = {
 0:'FM',
 1:'OQPSK',
 2:'BPSK',
 3:'8PSK',
 4:'AM-SSB-SC',
 5:'4ASK',
 6:'16PSK',
 7:'AM-DSB-SC', 
 8:'QPSK', 
 9:'OOK',
}

# class to name
def classToName(cls):
    return index_class[cls]

def onehotToNum(dataset):
    output = np.array(np.argmax(dataset, axis = 1), ndmin = 2).T
    return output

In [10]:
train_Y_hat = model.predict(X_train, batch_size=1024)
trainout = onehotToNum(train_Y_hat)
total = 30000
traincorrect = 0
traincorrect += (trainout == Y_number).sum().item()

print(train_Y_hat.shape)
print('Train Accuracy of the model over the 30000 train modulations: {} %'.format(100 * traincorrect / total)) 

(30000, 10)
Train Accuracy of the model over the 30000 train modulations: 65.36666666666666 %


In [11]:
test_Y_hat = model.predict(X_test, batch_size=1024)
testout = onehotToNum(test_Y_hat)

predictionlist = testout.tolist()
names = []

for line in predictionlist:
    names.append(classToName(line[0]))
    
outputs = np.asarray(names, dtype = str)
print(outputs)
categories = pd.DataFrame(outputs, columns = ['Category'])       
categories.to_csv('outputs.csv')

['OOK' 'OOK' 'BPSK' ... 'AM-SSB-SC' 'AM-DSB-SC' 'OOK']
